# Temporal Network Basics

In [1]:
import pathpy as pp

In [2]:
tn = pp.TemporalNetwork(directed=True)

In [3]:
tn.add_edge('a', 'b', timestamp=1, color='red')
tn.add_edge('b', 'c', timestamp=2)
tn.add_edge('c', 'd', timestamp=4, color='green')
tn.add_edge('d', 'f', timestamp=4)
tn.add_edge('a', 'f', timestamp=6)
tn.add_edge('c', 'f', start=7, duration=3)
tn.add_edge('b', 'f', start=7, end=11)
print(tn)

Uid:			0x2317fd5ccc0
Type:			TemporalNetwork
Directed:		True
Multi-Edges:		False
Number of unique nodes:	5
Number of unique edges:	7
Number of temp nodes:	5
Number of temp edges:	7
Observation period:	1 - 11
Observation length:	10


We can visualize a temporal network as follows:

In [4]:
tn.plot()

In a temporal network, we can iterate through the node objects just like in a regular network. The obejcts are instances of `TemporalNode` though:

In [5]:
for v in tn.nodes:
    print(v)

Uid:		a
Type:		TemporalNode

Uid:		b
Type:		TemporalNode

Uid:		c
Type:		TemporalNode

Uid:		d
Type:		TemporalNode

Uid:		f
Type:		TemporalNode



We can also use the `tnodes` iterator, which returns tuples `(start, end, v)` where `start` is the time at which the node appears in the network, `end` is the time at which the node disappears, and `v` is the uid of the `TemporalNode` object in the temporal network `nodes` collection.

In [6]:
for start, end, v in tn.tnodes:
    print(start, end, v)

-inf inf a
-inf inf b
-inf inf c
-inf inf d
-inf inf f


We can do the same with temporal edges using the `tedges` iterator. As you see below, an edge that is created using a `timestamp` will last until the next timestamp by default:

In [7]:
for start, end, e in tn.tedges:
    print(start, end, e)

1 2.0 0x2317fd8add8
2 3.0 0x2317fd8ada0
4 5.0 0x2312aac6e80
4 5.0 0x2317fd8afd0
6 7.0 0x2312aac6d30
7 11 0x2317fd90208
7 inf 0x2317fd90128


We can use the following method to turn a temporal network into a static network. In this case, alll edges and node ocurring at any time during the evolution will be aggregated:

In [8]:
n = pp.Network.from_temporal_network(tn)
n.plot()

We can also limit the time window within which the temporal nodes and edges are aggregated. The min_time is inclusive, while the max_time is an exclusive boundary, i.e. the following will generate a static network containing nodes and edges that exist at times t=1 and t=2

In [9]:
n = pp.Network.from_temporal_network(tn, min_time=1, max_time=3)
n.plot()

We can use the `RollingTimeWindow` iterator in `pathpy.algorithms` to generate a sequence of time-slice graphs. The width of the aggregation window is controlled by the parameter `window_size`. In each iteration, the current start time of the rolling window is increment by the `step_size`. This allows to create overlapping time windows. The iterator will automatically start at the first time stamp and it will end in the last time stamp for which there is a node or edge:

In [10]:
time_slices = [n for n in pp.algorithms.RollingTimeWindow(tn, window_size=1, step_size=1)]

We can plot the sequence of time slice graphs:

In [11]:
time_slices[0].plot()

In [12]:
time_slices[1].plot()

In [13]:
time_slices[2].plot()

In [14]:
time_slices[3].plot()

In [15]:
time_slices[4].plot()

In [16]:
time_slices[5].plot()

In [17]:
time_slices[6].plot()

In [18]:
time_slices[7].plot()

In [19]:
time_slices[8].plot()

In [20]:
time_slices[9].plot()

If we prefer a dictionary, where the keys are the start time of the time windows for which time slice graphs have been generated, we can use the following:

In [21]:
time_slices = {time[0]: n for n, time in pp.algorithms.RollingTimeWindow(tn, window_size=1, step_size=1, return_window=True)}

We can now access the time-slice graph for time stamp t=4 as follows:

In [22]:
time_slices[4].plot()

Let's now try this with a real data set:

In [23]:
tn = pp.io.graphtool.read_netzschleuder_network('sp_high_school_new', '2011')
print(tn)

Uid:			0x2310cdaeba8
Type:			TemporalNetwork
Directed:		False
Multi-Edges:		False
Number of unique nodes:	126
Number of unique edges:	1710
Number of temp nodes:	126
Number of temp edges:	28540
Observation period:	54120 - 326451.0
Observation length:	272331.0

Network attributes
------------------
name:	sp_high_school_new (2011)
description:	These datasets contain the temporal network of contacts between students in a high school in Marseilles, France. The first dataset gives the contacts of the students of three classes during 4 days in Dec. 2011, and the second corresponds to the contacts of the students of 5 classes during 7 days (from a Monday to the Tuesday of the following week) in Nov. 2012.[^icon]

Each Contact list file contains a tab-separated list representing the active contacts during 20-second intervals of the data collection. Each line has the form “t i j Ci Cj”, where i and j are the anonymous IDs of the persons in contact, Ci and Cj are their classes, and the interval d

To aggregate this temporal network into hourly time slice graphs, we can use the following:

In [25]:
time_slices = {time[0]: n for n, time in pp.algorithms.RollingTimeWindow(tn, window_size=3600, step_size=3600, return_window=True)}

The `2011` data set contains data on approx. 4 days, which we can confirm as follows:

In [33]:
len(time_slices)/24

3.125

In [35]:
time_slices[54120].plot()

In [36]:
time_slices[54120+3600].plot()